In [1]:
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import fuzz
import textacy

In [2]:
df = pd.read_csv('./Data/processedTrain.csv')
default =pd.read_csv('./Data/train.csv') 

In [3]:
df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate,q1,q2
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,3,4,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0,story kohinoor koh noor diamond,would happen indian government steal kohinoor ...
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,increase speed -PRON- internet connection use vpn,internet speed increase hack dns
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math is div...,0,mentally lonely solve -PRON-,find remainder math 23 24 math divide 24 23
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [4]:
## return average and amount of numbers
def getAvgNumber(x):
    x = str(x)
    nums =  re.findall('\d+', x)
    nums = np.array(nums)
    nums = nums.astype(np.float)
    if(len(nums)==0):
        return 0
    
    return sum(nums)/len(nums)

def getCountNumber(x):
    x = str(x)
    nums =  re.findall('\d+', x)
    nums = np.array(nums)
    nums = nums.astype(np.float)
    if(len(nums)==0):
        return 0
    return len(nums)
    
def getCharCount(x):
    x = str(x)
    return len(x)
def getSentCount(x):
    x = str(x)
    return x.count('.') + x.count('!') + x.count('?')

def getWordCount(x):
    x = str(x)
    return x.count(" ") + 1



In [5]:
avgNum1= df.q1.apply(lambda x:getAvgNumber(x))
avgNum2= df.q2.apply(lambda x:getAvgNumber(x))
df['avgNumDiff'] = abs(avgNum1-avgNum2)

In [6]:
countNum1= df.q1.apply(lambda x:getCountNumber(x))
countNum2 = df.q2.apply(lambda x:getCountNumber(x))
df['countNumDiff'] = abs(countNum1-countNum2)

In [7]:
charCount1= df.q1.apply(lambda x:getCharCount(x))
charCount2= df.q2.apply(lambda x:getCharCount(x))
df['charCountDiff'] = abs(charCount1-charCount2)

In [8]:
wordCount1 = df.q1.apply(lambda x:getWordCount(x))
wordCount2 = df.q2.apply(lambda x:getWordCount(x))
df['wordCountDiff'] = abs(wordCount1-wordCount2)

In [9]:
sentCount1= default.question1.apply(lambda x:getSentCount(x))
sentCount2= default.question2.apply(lambda x:getSentCount(x))
df['sentCountDiff'] = abs(sentCount1-sentCount2)

In [10]:
#https://pypi.python.org/pypi/fuzzywuzzy

df['simpleRatio'] = df.apply(lambda x: fuzz.ratio(str(x['q1']),str(x['q2'])),axis=1)
df['tokenSetRatio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['q1']),str(x['q2'])),axis=1)
df['tokenSortRatio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['q1']),str(x['q2'])),axis=1)
df['partialRatio'] = df.apply(lambda x: fuzz.partial_ratio(str(x['q1']),str(x['q2'])),axis=1)


In [11]:
#Textacy features
df['jaro_winkler'] = df.apply(lambda x: textacy.similarity.jaro_winkler(str(x['q1']),str(x['q2'])),axis=1)
df['levenshtein']  = df.apply(lambda x: textacy.similarity.levenshtein(str(x['q1']),str(x['q2'])),axis=1)
df['hamming']  = df.apply(lambda x: textacy.similarity.hamming(str(x['q1']),str(x['q2'])),axis=1)
df['jaccard']  = df.apply(lambda x: textacy.similarity.jaccard(str(x['q1']),str(x['q2'])),axis=1)

In [12]:
df.to_csv('./Data/simpleFeatures.csv', index = False)